In [31]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()
"""
tf.function
Compiles a function into a callable TensorFlow graph.
被装饰的这个方法会被tensorflow解析，将这里的运算逻辑放入graph中
tensorflow是定义与运算分离。
可以看到下面的exec只会执行一次，这时实际是在trace阶段（追踪数据流向轨迹、定义或者说编译）

先看下面的两个f1，说明trace阶段是在第一次调用是做的
而第二次调用的时候，实际上就只执行了f1中的图计算部分，即生成一个张量以及循环的三次write

再看f2部分，可以看到l是有3个值，而且只是普通的没有值的tensor
这是因为这个f2里除了一个循环之外，并没有真正的图计算的op，因此没有读取到值

f3里只会执行一次，这也是因为除了循环之外没有真正的图计算op，图里只知道有一个循环
所以结果只是一个while/add:0
但至于为什么f2的循环会执行三次，我猜只可能是lenx和直接对x循环的差别吧
"""


@tf.function
def f1(x):
  print("exec 1")
  ta = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
  for i in range(len(x)):
    print(i)
    ta = ta.write(i, x[i] + 1)
  return ta.stack()
f1(tf.constant([1, 2, 3]))
f1(tf.constant([1, 2, 3]))

l = []
@tf.function
def f2(x):
  print("exec 1")
  for i in range(len(x)):
    print(i) # 那这个为啥会执行三次啊
    l.append(x[i])
f2(tf.constant([1, 2, 3]))
print(l)

l=[]
@tf.function
def f3(x):
  for i in x:
    l.append(i + 1)    # Caution! Will only happen once when tracing
f3(tf.constant([1, 2, 3]))
print(l)
f3(tf.constant([1, 2, 3]))
print(l)

exec 1
0
1
2
exec 1
0
1
2
[<tf.Tensor 'strided_slice:0' shape=() dtype=int32>, <tf.Tensor 'strided_slice_1:0' shape=() dtype=int32>, <tf.Tensor 'strided_slice_2:0' shape=() dtype=int32>]
[<tf.Tensor 'while/add:0' shape=() dtype=int32>]
[<tf.Tensor 'while/add:0' shape=() dtype=int32>]


In [3]:
"""
讲解tf.reshape
这个reshape就是重新排序，其中-1代表不指定，tf自动计算出来
reshape的规则是，以最后一行作为讲解
原始tensor是2,4,6要变为2,2,3,4；
那就先从最后一个轴(0,0,:)取前4个数组成一个有4个数的一维向量；
然后再从(0,0,4:6)取出4个数，发现不够，那再从(0,1,:2)取两个凑够4个，组成第二个有4个数的一维向量；
重复上述过程直到构建了三个有4个数字的一维向量。就构成了一个结果的(0,0,0,:)
反复上述过程
"""
import tensorflow as tf
t = tf.reshape(tf.range(0,48),[2,4,6])
print(t)
print(tf.reshape(t, [2,4,-1,2]))
print(tf.reshape(t, [2,-1,4,2]))
print(tf.reshape(t, [2,2,3,4]))

tf.Tensor(
[[[ 0  1  2  3  4  5]
  [ 6  7  8  9 10 11]
  [12 13 14 15 16 17]
  [18 19 20 21 22 23]]

 [[24 25 26 27 28 29]
  [30 31 32 33 34 35]
  [36 37 38 39 40 41]
  [42 43 44 45 46 47]]], shape=(2, 4, 6), dtype=int32)
tf.Tensor(
[[[[ 0  1]
   [ 2  3]
   [ 4  5]]

  [[ 6  7]
   [ 8  9]
   [10 11]]

  [[12 13]
   [14 15]
   [16 17]]

  [[18 19]
   [20 21]
   [22 23]]]


 [[[24 25]
   [26 27]
   [28 29]]

  [[30 31]
   [32 33]
   [34 35]]

  [[36 37]
   [38 39]
   [40 41]]

  [[42 43]
   [44 45]
   [46 47]]]], shape=(2, 4, 3, 2), dtype=int32)
tf.Tensor(
[[[[ 0  1]
   [ 2  3]
   [ 4  5]
   [ 6  7]]

  [[ 8  9]
   [10 11]
   [12 13]
   [14 15]]

  [[16 17]
   [18 19]
   [20 21]
   [22 23]]]


 [[[24 25]
   [26 27]
   [28 29]
   [30 31]]

  [[32 33]
   [34 35]
   [36 37]
   [38 39]]

  [[40 41]
   [42 43]
   [44 45]
   [46 47]]]], shape=(2, 3, 4, 2), dtype=int32)
tf.Tensor(
[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]]

  [[12 13 14 15]
   [16 17 18 19]
   [20 21 22 23]]]


 [[[24 

In [19]:
"""
讲解tf.transpose
第0维没动，最后两个维度翻转，翻转结果为2，1；
也就是把原本第1个轴上相同位置的元素构成一个维度作为新tensor的最后一个维度
"""
# x = tf.constant([[[ 1,  2,  3],
#                   [ 4,  5,  6]],
#                  [[ 7,  8,  9],
#                   [10, 11, 12]]])
# print(x)
# tf.transpose(x, perm=[0, 2, 1])

t = tf.reshape(tf.range(0,48),[2,3,2,4])
print(t)
trans_t=tf.transpose(t,perm=[0,2,3,1])
print(trans_t)
"""
对于这些列表套列表的高纬度数组，不要只认为他们是列表套列表，更要理解成他们是由不同的轴组成的，每个轴上都有一串数字，可以看做一维向量
例如t[0,0,:,0]，就代表着当0轴1轴3轴都取值为0的时候，2轴的一串数字
而如果perm=[0,2,3,1]，就代表着，原本的2轴被挪到了新的1轴，原本3轴对应新2轴，原本1轴对应新3轴，所以才有最后的等式关系
"""
print(t[0,:,0,0])
print(t[0,0,:,0])
print(t[0,0,:,1])
# print(t[0,:,0,2])
# print(t[0,:,0,3])


print(t[0,0,:,0]==trans_t[0,:,0,0])
print(t[0,0,:,1]==trans_t[0,:,1,0])



tf.Tensor(
[[[[ 0  1  2  3]
   [ 4  5  6  7]]

  [[ 8  9 10 11]
   [12 13 14 15]]

  [[16 17 18 19]
   [20 21 22 23]]]


 [[[24 25 26 27]
   [28 29 30 31]]

  [[32 33 34 35]
   [36 37 38 39]]

  [[40 41 42 43]
   [44 45 46 47]]]], shape=(2, 3, 2, 4), dtype=int32)
tf.Tensor(
[[[[ 0  8 16]
   [ 1  9 17]
   [ 2 10 18]
   [ 3 11 19]]

  [[ 4 12 20]
   [ 5 13 21]
   [ 6 14 22]
   [ 7 15 23]]]


 [[[24 32 40]
   [25 33 41]
   [26 34 42]
   [27 35 43]]

  [[28 36 44]
   [29 37 45]
   [30 38 46]
   [31 39 47]]]], shape=(2, 2, 4, 3), dtype=int32)
tf.Tensor([ 0  8 16], shape=(3,), dtype=int32)
tf.Tensor([0 4], shape=(2,), dtype=int32)
tf.Tensor([1 5], shape=(2,), dtype=int32)
tf.Tensor([ True  True], shape=(2,), dtype=bool)
tf.Tensor([ True  True], shape=(2,), dtype=bool)
